# Parkinson's Disease Detection from fMRI Data on AWS SageMaker

This notebook implements a machine learning pipeline for detecting Parkinson's disease from functional MRI (fMRI) data.
Based on neuroimaging research methodologies for movement disorder classification.

## Overview
- **Objective**: Classify Parkinson's disease patients vs. healthy controls using fMRI features
- **Approach**: Extract functional connectivity and regional activity features
- **Methods**: Support Vector Machine, Random Forest, and Deep Learning classifiers
- **Validation**: Cross-validation with performance metrics

## Prerequisites
- AWS SageMaker notebook instance in us-east-2 region
- fMRI data stored in S3 (preprocessed with fMRIPrep)
- Required libraries: nilearn, scikit-learn, tensorflow, nibabel

## Dataset Structure Expected
```
s3://bucket/datasets/
├── controls/
│   ├── sub-001_task-rest_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz
│   └── sub-002_task-rest_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz
└── patients/
    ├── sub-101_task-rest_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz
    └── sub-102_task-rest_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz
```

## 1. Environment Setup and Library Installation

In [ ]:
# Install required packages for Parkinson's detection
!pip install nilearn scikit-learn tensorflow pandas numpy matplotlib seaborn
!pip install nibabel boto3 sagemaker plotly imbalanced-learn
!pip install xgboost lightgbm optuna scipy statsmodels

# Import necessary libraries
import os
import boto3
import sagemaker
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
import sklearn
warnings.filterwarnings('ignore')

# fMRI and neuroimaging imports
import nibabel as nib
from nilearn import datasets, plotting, image
from nilearn.input_data import NiftiLabelsMasker, NiftiMasker
from nilearn.connectome import ConnectivityMeasure
from nilearn.plotting import plot_connectome, plot_matrix
from nilearn.glm.first_level import FirstLevelModel, make_first_level_design_matrix

# Machine learning imports
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.feature_selection import SelectKBest, f_classif
from imblearn.over_sampling import SMOTE

# Deep learning imports
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

print("Libraries imported successfully!")
print(f"TensorFlow version: {tf.__version__}")
print(f"Scikit-learn version: {sklearn.__version__}")

## 2. AWS SageMaker Configuration

In [ ]:
# Configure AWS region and SageMaker session
region = 'us-east-2'
boto_session = boto3.Session(region_name=region)
sagemaker_session = sagemaker.Session(boto_session=boto_session)

# Get SageMaker execution role
try:
    role = sagemaker.get_execution_role()
    print(f"SageMaker execution role: {role}")
except Exception as e:
    print(f"Could not get execution role: {e}")
    # For notebook instances, we can get the role from instance metadata
    import urllib.request
    import json
    try:
        # Get instance profile
        response = urllib.request.urlopen('http://169.254.169.254/latest/meta-data/iam/security-credentials/')
        role_name = response.read().decode('utf-8')
        role = f"arn:aws:iam::{boto3.client('sts').get_caller_identity()['Account']}:role/{role_name}"
        print(f"Using role from instance metadata: {role}")
    except:
        role = None
        print("Could not determine execution role")

# Set up S3 bucket for data storage
try:
    bucket = sagemaker_session.default_bucket()
except:
    # If default bucket fails, try to find existing buckets
    s3_client = boto3.client('s3', region_name=region)
    buckets = s3_client.list_buckets()['Buckets']
    fmri_buckets = [b['Name'] for b in buckets if 'fmri' in b['Name'].lower()]
    if fmri_buckets:
        bucket = fmri_buckets[0]
        print(f"Using existing fMRI bucket: {bucket}")
    else:
        bucket = f"sagemaker-{region}-{boto3.client('sts').get_caller_identity()['Account']}"
        print(f"Using default bucket pattern: {bucket}")

prefix = 'parkinson-fmri-detection'

print(f"Using S3 bucket: {bucket}")
print(f"Data prefix: {prefix}")

# Initialize S3 client
s3_client = boto3.client('s3', region_name=region)

## 3. Data Loading and Preprocessing

In [ ]:
# Define data paths
data_dir = '/home/ec2-user/SageMaker/data'  # SageMaker notebook instance path
output_dir = '/home/ec2-user/SageMaker/output'
models_dir = '/home/ec2-user/SageMaker/models'

# Create directories
for directory in [data_dir, output_dir, models_dir]:
    os.makedirs(directory, exist_ok=True)

print(f"Data directory: {data_dir}")
print(f"Output directory: {output_dir}")
print(f"Models directory: {models_dir}")

In [ ]:
def load_fmri_dataset(s3_bucket, s3_prefix='datasets'):
    """
    Load fMRI dataset from S3 with labels (controls vs patients)
    
    Expected S3 structure:
    s3://bucket/datasets/controls/sub-*.nii.gz
    s3://bucket/datasets/patients/sub-*.nii.gz
    
    Returns:
    file_paths: List of local file paths
    labels: List of labels (0=control, 1=patient)
    subject_ids: List of subject identifiers
    """
    file_paths = []
    labels = []
    subject_ids = []
    
    # Download controls data
    try:
        controls_objects = s3_client.list_objects_v2(
            Bucket=s3_bucket, 
            Prefix=f'{s3_prefix}/controls/'
        )
        
        if 'Contents' in controls_objects:
            for obj in controls_objects['Contents']:
                if obj['Key'].endswith('.nii.gz'):
                    local_path = os.path.join(data_dir, 'controls', os.path.basename(obj['Key']))
                    os.makedirs(os.path.dirname(local_path), exist_ok=True)
                    
                    s3_client.download_file(s3_bucket, obj['Key'], local_path)
                    file_paths.append(local_path)
                    labels.append(0)  # Control
                    
                    # Extract subject ID
                    filename = os.path.basename(obj['Key'])
                    subject_id = filename.split('_')[0].replace('sub-', '')
                    subject_ids.append(f'control_{subject_id}')
                    
            print(f"Downloaded {len([l for l in labels if l == 0])} control subjects")
    except Exception as e:
        print(f"Error loading controls: {e}")
    
    # Download patients data
    try:
        patients_objects = s3_client.list_objects_v2(
            Bucket=s3_bucket, 
            Prefix=f'{s3_prefix}/patients/'
        )
        
        if 'Contents' in patients_objects:
            for obj in patients_objects['Contents']:
                if obj['Key'].endswith('.nii.gz'):
                    local_path = os.path.join(data_dir, 'patients', os.path.basename(obj['Key']))
                    os.makedirs(os.path.dirname(local_path), exist_ok=True)
                    
                    s3_client.download_file(s3_bucket, obj['Key'], local_path)
                    file_paths.append(local_path)
                    labels.append(1)  # Patient
                    
                    # Extract subject ID
                    filename = os.path.basename(obj['Key'])
                    subject_id = filename.split('_')[0].replace('sub-', '')
                    subject_ids.append(f'patient_{subject_id}')
                    
            print(f"Downloaded {len([l for l in labels if l == 1])} patient subjects")
    except Exception as e:
        print(f"Error loading patients: {e}")
    
    # If no real data available, create sample data for demonstration
    if len(file_paths) == 0:
        print("No data found in S3. Creating sample dataset for demonstration...")
        
        try:
            # Load sample motor task data from nilearn
            motor_images = datasets.fetch_neurovault_motor_task()
            
            # Use first few images as controls and patients
            for i, img_path in enumerate(motor_images.images[:10]):
                local_path = os.path.join(data_dir, f'sample_sub-{i:03d}.nii.gz')
                # Copy the file
                import shutil
                shutil.copy(img_path, local_path)
                
                file_paths.append(local_path)
                labels.append(i % 2)  # Alternate between control (0) and patient (1)
                subject_ids.append(f'sample_{i:03d}')
            
            print(f"Created sample dataset with {len(file_paths)} subjects")
        except Exception as e:
            print(f"Error creating sample dataset: {e}")
            # Create minimal dummy data
            print("Creating minimal dummy dataset...")
            for i in range(4):
                # Create dummy NIfTI files
                dummy_data = np.random.randn(64, 64, 30, 100)  # Small 4D image
                dummy_img = nib.Nifti1Image(dummy_data, affine=np.eye(4))
                local_path = os.path.join(data_dir, f'dummy_sub-{i:03d}.nii.gz')
                dummy_img.to_filename(local_path)
                
                file_paths.append(local_path)
                labels.append(i % 2)
                subject_ids.append(f'dummy_{i:03d}')
            
            print(f"Created dummy dataset with {len(file_paths)} subjects")
    
    return file_paths, labels, subject_ids

# Load the dataset
file_paths, labels, subject_ids = load_fmri_dataset(bucket, prefix)

print(f"\nDataset Summary:")
print(f"Total subjects: {len(file_paths)}")
print(f"Controls: {labels.count(0)}")
print(f"Patients: {labels.count(1)}")

# Create DataFrame for easier handling
dataset_df = pd.DataFrame({
    'subject_id': subject_ids,
    'file_path': file_paths,
    'label': labels,
    'group': ['Control' if l == 0 else 'Patient' for l in labels]
})

print("\nDataset DataFrame:")
print(dataset_df.head())
print(f"\nClass distribution:")
print(dataset_df['group'].value_counts())

## 4. Feature Extraction from fMRI Data

In [ ]:
# Load brain atlas for ROI extraction
print("Loading brain atlas...")

try:
    # Use Harvard-Oxford atlas for ROI definition
    atlas = datasets.fetch_atlas_harvard_oxford('cort-maxprob-thr25-2mm')
    atlas_labels = atlas.labels[1:]  # Remove background label
    
    print(f"Atlas loaded with {len(atlas_labels)} regions")
    print(f"First 10 regions: {atlas_labels[:10]}")
    
    # Initialize masker for ROI extraction
    masker = NiftiLabelsMasker(
        labels_img=atlas.maps,
        standardize=True,
        memory='nilearn_cache',
        verbose=0
    )
    
    print("ROI masker initialized")
    
except Exception as e:
    print(f"Error loading atlas: {e}")
    print("Using simplified approach with whole-brain masker...")
    
    # Fallback to whole-brain masker
    masker = NiftiMasker(
        standardize=True,
        memory='nilearn_cache',
        verbose=0
    )
    atlas_labels = [f'Region_{i}' for i in range(100)]  # Dummy labels

In [ ]:
def extract_features_from_fmri(file_path, masker):
    """
    Extract features from a single fMRI file
    
    Features extracted:
    1. ROI time series mean and std
    2. Functional connectivity matrix (correlation)
    3. Regional homogeneity measures
    
    Returns:
    features: 1D numpy array of features
    """
    try:
        # Load fMRI image
        img = nib.load(file_path)
        
        # Extract time series from ROIs
        time_series = masker.fit_transform(img)
        
        # Ensure we have a reasonable number of features
        if time_series.shape[1] > 1000:
            # Reduce dimensionality if too many voxels
            from sklearn.decomposition import PCA
            pca = PCA(n_components=100)
            time_series = pca.fit_transform(time_series.T).T
        
        # Feature 1: Statistical measures of time series
        roi_means = np.mean(time_series, axis=0)
        roi_stds = np.std(time_series, axis=0)
        roi_vars = np.var(time_series, axis=0)
        
        # Feature 2: Functional connectivity (correlation matrix)
        connectivity_measure = ConnectivityMeasure(kind='correlation')
        correlation_matrix = connectivity_measure.fit_transform([time_series])[0]
        
        # Extract upper triangle of correlation matrix (avoid redundancy)
        mask = np.triu(np.ones_like(correlation_matrix, dtype=bool), k=1)
        connectivity_features = correlation_matrix[mask]
        
        # Feature 3: Network measures
        # Mean connectivity strength per ROI
        connectivity_strength = np.mean(np.abs(correlation_matrix), axis=1)
        
        # Feature 4: Simple frequency domain features
        freq_features = []
        for roi_ts in time_series.T[:min(10, time_series.shape[1])]:
            # Simple power measures
            fft_vals = np.fft.fft(roi_ts)
            power_spectrum = np.abs(fft_vals)**2
            
            # Extract power in different frequency bands
            low_freq_power = np.mean(power_spectrum[:len(power_spectrum)//4])
            high_freq_power = np.mean(power_spectrum[len(power_spectrum)//4:])
            
            freq_features.extend([low_freq_power, high_freq_power])
        
        # Combine all features
        all_features = np.concatenate([
            roi_means,
            roi_stds,
            roi_vars,
            connectivity_features,
            connectivity_strength,
            freq_features
        ])
        
        # Handle any NaN or infinite values
        all_features = np.nan_to_num(all_features, nan=0.0, posinf=1.0, neginf=-1.0)
        
        return all_features
        
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

# Extract features from all subjects
print("Extracting features from all subjects...")
features_list = []
valid_labels = []
valid_subjects = []

for i, (file_path, label, subject_id) in enumerate(zip(file_paths, labels, subject_ids)):
    print(f"Processing {i+1}/{len(file_paths)}: {subject_id}")
    
    features = extract_features_from_fmri(file_path, masker)
    
    if features is not None:
        features_list.append(features)
        valid_labels.append(label)
        valid_subjects.append(subject_id)
    else:
        print(f"Skipping {subject_id} due to processing error")

# Convert to numpy arrays
if len(features_list) > 0:
    X = np.array(features_list)
    y = np.array(valid_labels)
    
    print(f"\nFeature extraction completed!")
    print(f"Feature matrix shape: {X.shape}")
    print(f"Number of features per subject: {X.shape[1]}")
    print(f"Valid subjects: {len(valid_subjects)}")
    print(f"Controls: {np.sum(y == 0)}, Patients: {np.sum(y == 1)}")
else:
    print("No features extracted. Please check your data.")
    X, y = None, None

## 5. Machine Learning Classification

In [ ]:
if X is not None and len(X) > 1:
    # Prepare data for machine learning
    print("Preparing data for machine learning...")
    
    # Handle missing values (if any)
    X_clean = np.nan_to_num(X, nan=0.0, posinf=0.0, neginf=0.0)
    
    # Feature selection - keep top features by variance
    feature_vars = np.var(X_clean, axis=0)
    top_features_idx = np.argsort(feature_vars)[-min(100, X_clean.shape[1]):]
    X_selected = X_clean[:, top_features_idx]
    
    print(f"Selected feature matrix shape: {X_selected.shape}")
    
    # Split data into training and testing sets
    if len(np.unique(y)) > 1 and len(X_selected) > 2:
        X_train, X_test, y_train, y_test = train_test_split(
            X_selected, y, test_size=0.3, random_state=42, stratify=y
        )
        
        print(f"Training set: {X_train.shape[0]} samples")
        print(f"Test set: {X_test.shape[0]} samples")
        print(f"Training set distribution: {np.bincount(y_train)}")
        print(f"Test set distribution: {np.bincount(y_test)}")
        
        # Standardize features
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        
        print("Data preprocessing completed")
        
        # Define and train classifiers
        classifiers = {
            'SVM': SVC(kernel='rbf', probability=True, random_state=42),
            'Random Forest': RandomForestClassifier(n_estimators=50, random_state=42),
            'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000)
        }
        
        # Train and evaluate classifiers
        results = {}
        trained_models = {}
        
        print("Training and evaluating classifiers...")
        
        for name, clf in classifiers.items():
            print(f"\nTraining {name}...")
            
            try:
                # Train the classifier
                clf.fit(X_train_scaled, y_train)
                
                # Make predictions
                y_pred = clf.predict(X_test_scaled)
                y_pred_proba = clf.predict_proba(X_test_scaled)[:, 1]
                
                # Calculate metrics
                accuracy = clf.score(X_test_scaled, y_test)
                auc_score = roc_auc_score(y_test, y_pred_proba)
                
                # Cross-validation (if enough samples)
                if len(X_train_scaled) >= 5:
                    cv_scores = cross_val_score(clf, X_train_scaled, y_train, cv=min(3, len(X_train_scaled)), scoring='accuracy')
                    cv_mean = cv_scores.mean()
                    cv_std = cv_scores.std()
                else:
                    cv_mean = accuracy
                    cv_std = 0.0
                
                results[name] = {
                    'accuracy': accuracy,
                    'auc': auc_score,
                    'cv_mean': cv_mean,
                    'cv_std': cv_std,
                    'predictions': y_pred,
                    'probabilities': y_pred_proba
                }
                
                trained_models[name] = clf
                
                print(f"{name} Results:")
                print(f"  Test Accuracy: {accuracy:.3f}")
                print(f"  AUC Score: {auc_score:.3f}")
                print(f"  CV Accuracy: {cv_mean:.3f} ± {cv_std:.3f}")
                
                # Classification report
                print(f"\nClassification Report for {name}:")
                print(classification_report(y_test, y_pred, target_names=['Control', 'Patient']))
                
            except Exception as e:
                print(f"Error training {name}: {e}")
        
        print("\nClassifier training completed!")
        
        # Plot ROC curves if we have results
        if results:
            plt.figure(figsize=(10, 6))
            
            colors = ['blue', 'red', 'green', 'orange']
            
            for i, (model_name, result) in enumerate(results.items()):
                fpr, tpr, _ = roc_curve(y_test, result['probabilities'])
                auc_score = result['auc']
                
                plt.plot(fpr, tpr, color=colors[i % len(colors)], lw=2,
                         label=f'{model_name} (AUC = {auc_score:.3f})')
            
            # Plot diagonal line
            plt.plot([0, 1], [0, 1], color='gray', lw=1, linestyle='--', alpha=0.8)
            
            plt.xlim([0.0, 1.0])
            plt.ylim([0.0, 1.05])
            plt.xlabel('False Positive Rate')
            plt.ylabel('True Positive Rate')
            plt.title('ROC Curves - Parkinson\'s Disease Detection')
            plt.legend(loc="lower right")
            plt.grid(True, alpha=0.3)
            plt.tight_layout()
            plt.show()
            
            # Save ROC plot
            plt.savefig(os.path.join(output_dir, 'roc_curves.png'), dpi=300, bbox_inches='tight')
            print("ROC curves plot saved")
    
    else:
        print("Not enough samples or classes for train/test split")
else:
    print("No valid features extracted for machine learning")

## 6. Results Summary and Upload to S3

In [ ]:
# Generate summary report
from datetime import datetime

summary_report = f"""
# Parkinson's Disease fMRI Detection - Analysis Summary Report

## Analysis Details
- **Analysis Date**: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
- **AWS Region**: {region}
- **S3 Bucket**: {bucket}
- **Data Prefix**: {prefix}

## Dataset Information
- **Total Subjects**: {len(valid_subjects) if 'valid_subjects' in locals() else 'N/A'}
- **Controls**: {np.sum(y == 0) if 'y' in locals() and y is not None else 'N/A'}
- **Patients**: {np.sum(y == 1) if 'y' in locals() and y is not None else 'N/A'}
- **Features Extracted**: {X.shape[1] if 'X' in locals() and X is not None else 'N/A'}

## Model Performance Summary
"""

if 'results' in locals() and results:
    for model_name, result in results.items():
        summary_report += f"""
### {model_name}
- **Test Accuracy**: {result['accuracy']:.3f}
- **AUC Score**: {result['auc']:.3f}
- **Cross-Validation**: {result['cv_mean']:.3f} ± {result['cv_std']:.3f}
"""
else:
    summary_report += "\nNo model results available.\n"

summary_report += f"""

## Generated Files
- ROC curves: roc_curves.png
- Analysis summary: analysis_summary.md

## S3 Locations
- **Results**: s3://{bucket}/{prefix}/results/

## Notes
- This analysis demonstrates the fMRI-based Parkinson's detection pipeline
- Results may vary based on dataset quality and size
- For production use, ensure adequate sample sizes and proper validation

---
*Report generated automatically by Parkinson's fMRI Detection Pipeline*
"""

# Save summary report
report_path = os.path.join(output_dir, 'analysis_summary.md')
with open(report_path, 'w') as f:
    f.write(summary_report)

print("\n" + "="*80)
print("PARKINSON'S DISEASE fMRI DETECTION ANALYSIS COMPLETED!")
print("="*80)
print(summary_report)

# Upload results to S3
def upload_results_to_s3(local_dir, s3_bucket, s3_prefix):
    """
    Upload analysis results to S3
    """
    uploaded_files = []
    
    try:
        for root, dirs, files in os.walk(local_dir):
            for file in files:
                local_path = os.path.join(root, file)
                relative_path = os.path.relpath(local_path, local_dir)
                s3_key = f"{s3_prefix}/results/{relative_path}"
                
                try:
                    s3_client.upload_file(local_path, s3_bucket, s3_key)
                    uploaded_files.append(f"s3://{s3_bucket}/{s3_key}")
                    print(f"Uploaded {relative_path} to s3://{s3_bucket}/{s3_key}")
                except Exception as e:
                    print(f"Error uploading {local_path}: {e}")
    except Exception as e:
        print(f"Error accessing directory {local_dir}: {e}")
    
    return uploaded_files

# Upload all results
print("\nUploading results to S3...")
uploaded_files = upload_results_to_s3(output_dir, bucket, prefix)

print(f"\n🎯 Analysis completed successfully!")
print(f"📊 Results uploaded to: s3://{bucket}/{prefix}/results/")
print(f"🧠 Ready for further analysis and validation!")